# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
pwd

'C:\\Users\\Michael\\FinTech Bootcamp\\week_11\\homework\\Starter_Code'

In [5]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [6]:
#Dummy Encoding
df_binary_encoded = pd.get_dummies(df, columns=["home_ownership", "verification_status", "issue_d", "loan_status", "pymnt_plan", "initial_list_status", "next_pymnt_d", "application_type", "hardship_flag", "debt_settlement_flag"])
df_binary_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1


In [7]:
pwd

'C:\\Users\\Michael\\FinTech Bootcamp\\week_11\\homework\\Starter_Code'

In [8]:
file_path = Path("Resources/encoded_data.csv")
df_binary_encoded.to_csv(file_path, index=False)

In [9]:
# Create our features
X = df_binary_encoded

# Create our target
y = df[['loan_status']]

In [10]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.994958,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.070831,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [11]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [12]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [13]:
# Create the StandardScaler instance
from sklearn.preprocessing import LabelEncoder, StandardScaler
data_scaler = StandardScaler()

In [14]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
data_scaler.fit(X)

StandardScaler()

In [15]:
# Scale the training and testing data
loans_data_scaled = data_scaler.transform(X)
loans_data_scaled[:98]

array([[-0.60109272,  0.91798436, -0.36555835, ..., -0.40290386,
         0.        ,  0.        ],
       [ 0.80978726,  1.50182932,  1.55674724, ..., -0.40290386,
         0.        ,  0.        ],
       [ 0.32327692,  1.50182932,  0.17089176, ..., -0.40290386,
         0.        ,  0.        ],
       ...,
       [ 0.32327692, -0.8231618 ,  0.53314225, ..., -0.40290386,
         0.        ,  0.        ],
       [ 0.71248519, -0.07725312,  0.2016492 , ..., -0.40290386,
         0.        ,  0.        ],
       [-0.26053549,  0.23440789, -0.54020926, ..., -0.40290386,
         0.        ,  0.        ]])

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
import pydotplus
from IPython.display import Image

In [17]:
# Resample the training data with the BalancedRandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [18]:
# Calculated the balanced accuracy score
predictions = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)

In [19]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_loans_data_scaled = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)


In [20]:
# Print the imbalanced classification report
display(cm_loans_data_scaled)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

,Predicted 0,Predicted 1
Actual 0,92,0
Actual 1,0,17113


Accuracy Score : 1.0
Classification Report
              precision    recall  f1-score   support

   high_risk       1.00      1.00      1.00        92
    low_risk       1.00      1.00      1.00     17113

    accuracy                           1.00     17205
   macro avg       1.00      1.00      1.00     17205
weighted avg       1.00      1.00      1.00     17205



In [21]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.3564210543290855, 'loan_status_low_risk'),
 (0.3513707310530195, 'loan_status_high_risk'),
 (0.04345720406328645, 'total_rec_int'),
 (0.04262037400267863, 'last_pymnt_amnt'),
 (0.04153469586470624, 'total_pymnt'),
 (0.03537125339987224, 'total_pymnt_inv'),
 (0.03341805014669057, 'total_rec_prncp'),
 (0.005188967631171166, 'total_rec_late_fee'),
 (0.002828373920886656, 'issue_d_Mar-2019'),
 (0.0027727487321792016, 'out_prncp_inv'),
 (0.002746113960743083, 'out_prncp'),
 (0.002660448518417292, 'mo_sin_old_rev_tl_op'),
 (0.0024936915056501144, 'dti'),
 (0.0022820898629016513, 'bc_open_to_buy'),
 (0.0021097043585991443, 'revol_bal'),
 (0.0020811115735617546, 'num_actv_rev_tl'),
 (0.00205020201116371, 'total_bc_limit'),
 (0.002006533534282226, 'bc_util'),
 (0.0019941862213733604, 'max_bal_bc'),
 (0.001981133548121481, 'mo_sin_old_il_acct'),
 (0.0019090316995305373, 'installment'),
 (0.0018765474817814035, 'total_rev_hi_lim'),
 (0.001873025265381389, 'total_bal_ex_mort'),
 (0.00179774322

### Easy Ensemble Classifier

In [22]:
# Train the Classifier
# YOUR CODE HERE

In [23]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [24]:
# Display the confusion matrix
# YOUR CODE HERE

In [25]:
# Print the imbalanced classification report
# YOUR CODE HERE

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.